In [1]:
# autoload
%load_ext autoreload
%autoreload 2

# import necessary libraries
import numpy as np

# Load PGL libraries and start a PGL window
from pgl import pgl, pglTask, pglExperiment, pglParameter, pglParameterBlock
pgl = pgl()

# close any existing windows
pgl.cleanUp()

================================ pglBase: init =================================
(pgl) mglMetal error log can be viewed in MacOS Console app by searching for PROCESS mglMetal or in a terminal with:
      log stream --level info --process mglMetal
(pgl) To search for something specifc, e.g. messages from mglMovie:
      log stream --predicate 'eventMessage CONTAINS "mglMovie"' --style syslog --level info
(pgl:checkOS) Python version: 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:35:20) [Clang 16.0.6 ]
(pgl:checkOS) Running on MacBook Pro (MacBookPro18,3) with macOS version: 26.3
(pgl:checkOS) Apple M1 Pro Cores: 8 (6 Performance and 2 Efficiency) Memory: 32 GB
(pgl:checkOS) GPU: Apple M1 Pro (Built-In) 14 cores, Metal 4 support
(pgl:checkOS)   Color LCD [Main Display]: 3024 x 1964 Retina (Built-in Liquid Retina XDR Display) GammaTable size: 1024
(pglBase) Main library instance created
(pglBase:removeOrphanedSockets) No orphaned sockets found in /Users/justin/Library/Containe

In [ ]:
# Set up task class
class pglRandomDotTask(pglTask):
    
    ########################
    def __init__(self, pgl):
        super().__init__()
        
        # set task parameters, these will automatically be saved in the settings file
        self.settings.taskName = "Random Dot Motion Task"
        self.settings.seglen = [1, 0.5]
        
        # fixed parameters, these will automatically be saved in the settings file
        self.settings.fixedParameters = {
            'width':15,
            'height':10,
            'coherence':(0.1,1),
            'dir':np.arange(0,360,45)
        }        
        p = self.settings.fixedParameters

        # add parameters for coherence and direction
        coherence = pglParameter('coherence',p['coherence'])
        dir = pglParameter('dir',p['dir'])
        self.addParameter(pglParameterBlock([dir, coherence]))
        
        # initalize stimulus
        self.rdk = pgl.randomDots(width=p['width'], height=p['height'])
    ########################
    def updateScreen(self):
        if self.currentSegment==0:
            self.rdk.display(direction=self.currentParams['dir'], coherence=self.currentParams['coherence'], speed=5.0)

# initialize task
randomDotTask = pglRandomDotTask(pgl)



In [ ]:
# Set up experiment
e = pglExperiment(pgl, "window")
e.initScreen()

# add the random dot motion task
e.addTask(randomDotTask)

# and run the experiment
e.run()

In [ ]:
from pgl import pglStaircaseUpDown
# Set up task class
class pglFixationTaskLeftRight(pglTask):
    
    ########################
    def __init__(self, pgl, demo=False):
        super().__init__()
        
        # for demo make fixSie large, and slow dowin timing
        if demo:
            fixSize = 10.0
            slowDownFactor = 5
        else:
            fixSize = 1.0
            slowDownFactor = 1
        
        # task name
        self.settings.taskName = "Fixation Task Left Right"
        
        # keep fixed parameters in settings (so they get saved)
        self.settings.fixedParameters = {
            'fixSize':fixSize,
            'slowDownFactor':slowDownFactor
        }
        self.fixSize = self.settings.fixedParameters['fixSize']
        
        # segments. Fixation, Stimulus, Response
        self.settings.seglen = (slowDownFactor * np.array([0.5, 0.2, 3.0])).tolist()
        
        # add a parameter for which side of the fixation cross dims
        self.addParameter(pglParameter('side',(-1,1)))
        
        # initialize stairase
        self.staircase = pglStaircaseUpDown()
        self.staircase.startStaircase()
    
    ########################
    def startSegment(self, startTime):
        '''
        Start a segment.
        '''
        super().startSegment(startTime)
        # display stimulus only during first segment
        if self.currentSegment==0:
            # fixation cross starts out white
            self.horizontalColor = self.verticalColor = 1.0
            # get the current decrement value from the staircase
            self.decrement = self.staircase.get()
        elif self.currentSegment==1:
            # during the stimulus phase, the left and right sides are different colored
            self.leftColor = self.rightColor = 1.0
            # dim left or right side of fixation cross
            if self.currentParams['side']==-1:
                self.leftColor = 1-self.decrement
            else:
                self.rightColor = 1-self.decrement
        elif self.currentSegment==2:
            # during response phase, make vertical change color
            self.verticalColor = [0.0, 1.0, 1.0]

    ########################
    def updateScreen(self):
        
        # draw disc blocking stimulus
        self.pgl.arc(0, 0, 0, self.fixSize/2, 0, 2*np.pi, color=self.pgl.clearScreenColor)
        if self.currentSegment==1:
            # draw the left side
            self.pgl.line(-self.fixSize/2, 0, 0, 0, self.leftColor)
            # draw the right side
            self.pgl.line(0, 0, self.fixSize/2, 0, self.rightColor)
        else:
            # just draw the horizotnal line
            self.pgl.line(-self.fixSize/2, 0, self.fixSize/2, 0, self.horizontalColor)

        # draw the vertical line
        self.pgl.line(0, -self.fixSize/2, 0, self.fixSize/2, self.verticalColor)    
        
    ########################
    def handleSubjectResponse(self, responses, updateTime):
        # make sure there is only one response
        if len(responses) > 0 and self.currentSegment > 1:
            if len(responses)!= 1:
                print("(fixationTaskLeftRight) Multiple responses detected, ignoring.")
            else:
                response = responses[0]
                self.leftColor = self.rightColor = self.horizontalColor = self.verticalColor = [1.0, 0.0, 0.0]
                correct = False
                # check if response is correct
                if ((response==0 and self.currentParams['side']==-1) or
                    (response==1 and self.currentParams['side']==1)):
                    correct = True 
                    self.leftColor = self.rightColor = self.horizontalColor = self.verticalColor = [0.0, 1.0, 0.0]
                # update staircase
                self.staircase.update(self.decrement, correct)
                print(f"(fixationTaskLeftRight) Decrement {self.decrement}: {'correct' if correct else 'incorrect'}")

# initialize task
fixationTask = pglFixationTaskLeftRight(pgl)

# todo: If subject does not respond within time limit, treat as incorrect response
# todo: Only allow one response per trial
# todo: Restart at previous threshold
# todo: get background color from pgl

In [12]:
# Set up experiment
e = pglExperiment(pgl, "window")
e.initScreen()

# add the random dot motion task
e.addTask(randomDotTask)
e.addTask(fixationTask)


# and run the experiment
e.run()

(pglSettingsManager:loadSettings) Loading settings from '/Users/justin/.pgl/settings/window.json'.
(pglSettings) 'startOnVolumeTrigger' not found in JSON file /Users/justin/.pgl/settings/window.json, using default False
================================= pglBase:open =================================
(pglBase:open) Starting mglMetal application: /Users/justin/Library/Developer/Xcode/DerivedData/Build/Products/Debug/mglMetal.app
(pglBase:open) Using socket with address: /Users/justin/Library/Containers/gru.mglMetal/Data/pglMetal.socket.20260222_083409.LAcdiSj4Fh
(pgl:_pglComm) .Connected to: /Users/justin/Library/Containers/gru.mglMetal/Data/pglMetal.socket.20260222_083409.LAcdiSj4Fh
(pgl:_resolution:getResolution) Display 0/1: 1512x982 120Hz 32bits
(pglKeyboardMouse:start) Starting keyboard and mouse event listener.
(pglEventListener) Eating 7 keys: [18, 19, 20, 21, 49, 50, 53]
(pglImage:imageDelete) Deleting image 1 (653x51)
(pglExperiment:startPhase) Starting phase: 1/1
seglen:  ['1',

In [ ]:
e = pglExperiment(pgl, "window", subjectID="s1234", experimentName="My First Experiment")
e.addTask(fixationTask)
e.save()

In [9]:
help(pgl.arc)

Help on method arc in module pgl.pglDraw:

arc(x, y, innerRadius=0, outerRadius=1, startAngle=0, stopAngle=3.141592653589793, borderSize=0.1, color=None) method of pgl.pgl instance
    Draw arcs.

    Args:
        x (float or list): x coordinate(s) of the center of the arc(s).
        y (float or list): y coordinate(s) of the center of the arc(s).
        innerRadius (float): The inner radius of the arc.
        outerRadius (float): The outer radius of the arc.
        startAngle (float): The starting angle of the arc in radians.
        stopAngle (float): The stopping angle of the arc in radians.
        color (list or tuple, optional): RGB color values as a list or tuple of three floats in the range [0, 1].

    Returns:
        None

